#### Test Pipeline
This is a Julia Notebook to check the correctness of the C++ code. 

The notebook proceeds as follows: 
- Generate a synthetic instance
- Run the original implementation of the algorithm (pure Julia)
- Run the C++ implementation of the algorithm

#### Step 1: Data Generation

Generates a ground truth covariance matrix S of the form $I + \beta x_1 x_1^\top + \beta x_2 x_2^\top$, where 
- $x_1, x_2$ are $k$-sparse, with non-overlapping support 
- $\beta$ controls the signal-to-noise ratio

Then, samples $n$ multivariate normal observation from $\mathcal{N}(0,S)$ and constructs the empirical covariance matrix $\Sigma$

In [1]:
using Random, LinearAlgebra
Random.seed!(1234)

p = 10 #Dimension
r = 2 #Number of sparse PCs
k = 4 #Sparsity of each PC
β = 1 #Signal strength


x1 = zeros(p); x1[1:k] = sign.(rand(k) .- .5)
x2 = zeros(p); x2[(k+1):(k+k)] = sign.(rand(k) .- .5)

# x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap)] = sign.(rand(k_overlap) .- .5)
# x2[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)] = -x1[(2*k_nonoverlapping+1):(2*k_nonoverlapping+k_overlap_half)]
# x2[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)] = x1[(2*k_nonoverlapping+k_overlap_half+1):(2*k_nonoverlapping+k_overlap)]

@assert sum(abs.(x1) .> 0) == k
@assert sum(abs.(x2) .> 0) == k
@assert abs(dot(x1,x2)) ≤ 1e-10

shufflecoords = randperm(p)
x1 = x1[shufflecoords]; x2=x2[shufflecoords] 

x1 /= norm(x1); x2 /= norm(x2) 

S = β*x1*x1'+β*x2*x2'+ Matrix(1.0*I, p, p)
S = (S + S')/2

10×10 Matrix{Float64}:
  1.25   0.25  0.0  0.0   0.0    0.0    0.0   -0.25  -0.25   0.0
  0.25   1.25  0.0  0.0   0.0    0.0    0.0   -0.25  -0.25   0.0
  0.0    0.0   1.0  0.0   0.0    0.0    0.0    0.0    0.0    0.0
  0.0    0.0   0.0  1.0   0.0    0.0    0.0    0.0    0.0    0.0
  0.0    0.0   0.0  0.0   1.25   0.25  -0.25   0.0    0.0   -0.25
  0.0    0.0   0.0  0.0   0.25   1.25  -0.25   0.0    0.0   -0.25
  0.0    0.0   0.0  0.0  -0.25  -0.25   1.25   0.0    0.0    0.25
 -0.25  -0.25  0.0  0.0   0.0    0.0    0.0    1.25   0.25   0.0
 -0.25  -0.25  0.0  0.0   0.0    0.0    0.0    0.25   1.25   0.0
  0.0    0.0   0.0  0.0  -0.25  -0.25   0.25   0.0    0.0    1.25

In [2]:
n = 1000 #Sample size

Random.seed!(1234)

using Distributions
d = MvNormal(zeros(p), S)
X = rand(d, n) #p by N matrix of observations

Sn = cov(X') #Sample covariance matrix

10×10 Matrix{Float64}:
  1.19918      0.272751    -0.0275862   …  -0.255738   -0.238637    0.0827532
  0.272751     1.2777      -0.072279       -0.34593    -0.325634   -0.0808126
 -0.0275862   -0.072279     1.02957         0.0472149   0.0213239   0.0220759
  0.0227836    0.00966214   0.00435918     -0.0416068  -0.0176989  -0.0304168
 -0.0218518    0.0632956   -0.00482757     -0.0352677   0.0490125  -0.252779
  0.00376276   0.0444074   -0.0216567   …  -0.0519505  -0.0310439  -0.15713
 -0.00259936  -0.0351958   -0.0289323      -0.0595101  -0.0786064   0.267666
 -0.255738    -0.34593      0.0472149       1.34151     0.286398   -0.0679424
 -0.238637    -0.325634     0.0213239       0.286398    1.28691    -0.0201723
  0.0827532   -0.0808126    0.0220759      -0.0679424  -0.0201723   1.32953

In [3]:
# show(stdout, "text/plain", Sn)

In [4]:
# [x1 x2]

In [5]:
# [k, k]

#### Step 2: Julia benchmark

Applies the Julia code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [6]:
include("algorithm2.jl")

findmultPCs_deflation (generic function with 1 method)

In [7]:
ofv_best, violation_best, runtime, x_best = findmultPCs_deflation(Sn, r, [k,k]; numIters = 10, verbose = true, violation_tolerance = 1e-4 )

x_best

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern: [4, 4]

  Iteration |      Objective value |   Orthogonality Violation |       Time 
          1 |                0.351 |                  2.00e+00 |      0.771 


          2 |                0.351 |                  2.00e+00 |      1.044 
          3 |                0.351 |                  2.00e+00 |      1.044 
          4 |                0.351 |                  2.00e+00 |      1.044 
          5 |                0.314 |                  8.20e-01 |      1.045 
          6 |                0.336 |                  3.33e-16 |      1.045 
          7 |                0.336 |                  1.11e-16 |      1.045 
          8 |                0.336 |                  3.33e-16 |      1.045 
          9 |                0.336 |                  0.00e+00 |      1.046 


10×2 Matrix{Float64}:
  0.0       -0.421568
  0.0       -0.529844
  0.0        0.0
  0.0        0.0
 -0.458548   0.0
 -0.468789   0.0
  0.536703   0.0
  0.0        0.543248
  0.0        0.496414
  0.530962   0.0

In [8]:
[x1 x2]

10×2 Matrix{Float64}:
 -0.5   0.0
 -0.5   0.0
  0.0   0.0
  0.0   0.0
  0.0   0.5
  0.0   0.5
  0.0  -0.5
  0.5   0.0
  0.5   0.0
  0.0  -0.5

In [9]:
# show(stdout, "text/plain", x_best)

#### Step 3: R/C++ implementation

Applies the R code to $S$ (true covariance matrix) and $\Sigma$ (emprirical covariance matrix)

In [10]:
using RCall

R"""library(sPCAmPC)"""
# R"""
# library(devtools)
# reload(pkg = "sPCAmPC", quiet = FALSE)"""


RObject{StrSxp}
[1] "sPCAmPC"   "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     


In [11]:
Sn2 = round.(Sn, digits = 3)

10×10 Matrix{Float64}:
  1.199   0.273  -0.028   0.023  -0.022  …  -0.003  -0.256  -0.239   0.083
  0.273   1.278  -0.072   0.01    0.063     -0.035  -0.346  -0.326  -0.081
 -0.028  -0.072   1.03    0.004  -0.005     -0.029   0.047   0.021   0.022
  0.023   0.01    0.004   0.985  -0.016     -0.036  -0.042  -0.018  -0.03
 -0.022   0.063  -0.005  -0.016   1.219     -0.19   -0.035   0.049  -0.253
  0.004   0.044  -0.022   0.027   0.229  …  -0.265  -0.052  -0.031  -0.157
 -0.003  -0.035  -0.029  -0.036  -0.19       1.311  -0.06   -0.079   0.268
 -0.256  -0.346   0.047  -0.042  -0.035     -0.06    1.342   0.286  -0.068
 -0.239  -0.326   0.021  -0.018   0.049     -0.079   0.286   1.287  -0.02
  0.083  -0.081   0.022  -0.03   -0.253      0.268  -0.068  -0.02    1.33

In [18]:
@time R"""

TestMat <- $Sn2

results <- cpp_findmultPCs_deflation(TestMat, 2, c(4, 4), numIters=10)
"""

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern:  4 4
  Iteration |      Objective value |   Orthogonality Violation |       Time
          1 |                0.351 |                  2.00e+00 |      0.019
          2 |                0.351 |                  2.00e+00 |      0.035
          3 |                0.351 |                  2.00e+00 |      0.052
          4 |                0.351 |                  2.00e+00 |      0.074
          5 |                0.314 |                  8.19e-01 |      0.089
          6 |                0.336 |                  2.22e-16 |      0.109
          7 |                0.336 |                  3.33e-16 |      0.123
          8 |                0.336 |                  0.00e+00 |      0.136
          9 |                0.336 |                  4.44e-16 |      0.150
  0.150809 seconds (321 allocations: 10.070 KiB)


RObject{VecSxp}
$objective_value
[1] 4.118323

$orthogonality_violation
[1] 2.220446e-16

$runtime
[1] 0.15

$x_best
            [,1]       [,2]
 [1,]  0.0000000  0.4231478
 [2,]  0.0000000  0.5325307
 [3,]  0.0000000  0.0000000
 [4,]  0.0000000  0.0000000
 [5,] -0.4582342  0.0000000
 [6,] -0.4762564  0.0000000
 [7,]  0.5374278  0.0000000
 [8,]  0.0000000 -0.5385986
 [9,]  0.0000000 -0.4972610
[10,]  0.5238058  0.0000000



In [15]:
@time ofv_best, violation_best, runtime, x_best = findmultPCs_deflation(Sn, r, [k,k]; numIters = 10, verbose = true, violation_tolerance = 1e-4 )

x_best

---- Iterative deflation algorithm for sparse PCA with multiple PCs ---
Dimension: 10
Number of PCs: 2
Sparsity pattern: [4, 4]

  Iteration |      Objective value |   Orthogonality Violation |       Time 
          1 |                0.351 |                  2.00e+00 |      0.000 
          2 |                0.351 |                  2.00e+00 |      0.001 
          3 |                0.351 |                  2.00e+00 |      0.001 
          4 |                0.351 |                  2.00e+00 |      0.001 
          5 |                0.314 |                  8.20e-01 |      0.002 
          6 |                0.336 |                  3.33e-16 |      0.003 
          7 |                0.336 |                  1.11e-16 |      0.004 
          8 |                0.336 |                  3.33e-16 |      0.005 
          9 |                0.336 |                  2.22e-16 |      0.006 
  0.006895 seconds (34.23 k allocations: 2.758 MiB)


10×2 Matrix{Float64}:
  0.0       -0.421568
  0.0       -0.529844
  0.0        0.0
  0.0        0.0
  0.458548   0.0
  0.468789   0.0
 -0.536703   0.0
  0.0        0.543248
  0.0        0.496414
 -0.530962   0.0

In [13]:
[x1 x2]

10×2 Matrix{Float64}:
 -0.5   0.0
  0.0  -0.5
  0.0   0.5
  0.0  -0.5
  0.0   0.0
 -0.5   0.0
  0.0  -0.5
  0.0   0.0
  0.5   0.0
 -0.5   0.0